# Loading a semantic map using knowrob

This notebook demonstrates how to load a semantic map into KnowRob

### Importing KnowRob Modules

We will first initialize KnowRob

In [1]:
import json
try:
	# This case works in ros1 environments
	from knowrob.kb import *
except ImportError:
	# If the import fails, import the knowrob.so directly
	from knowrob import *
    
InitKnowRob()

[14:14:16.903] [info] [KnowRob] static initialization done.


### Setting Up Knowledge Base

Now we set up the knowledge base. See the second data-sources field, to see the path to an expected ontology. Add this ontology before running this cell.

In [2]:
# Sample dictionary to be converted to JSON
sample_dict = {
	"logging": {
		"console-sink": {"level": "debug"},
		"file-sink": {"level": "debug"}
	},
	"semantic-web": {
		"prefixes": [
			{"alias": "USD", "uri": "https://ease-crc.org/ont/USD.owl"}
		]
	},
	"data-sources": [
		{"path": "owl/test/USD.owl", "format": "rdf-xml"},
		{"path": "owl/test/house_4.owl", "format": "rdf-xml"}
	],
	"data-backends": [
		{
			"type": "MongoDB",
			"name": "mongodb",
			"host": "localhost",
			"port": 27017,
			"db": "test",
			"read-only": False
		}
	],
	"reasoner": []
}
# Convert the dictionary to a JSON string
json_str = json.dumps(sample_dict)
# Initialize the KnowledgeBase with the PropertyTree
kb = KnowledgeBase(json_str)

[14:14:16.917] [info] Using backend `mongodb` with type `MongoDB`.
[14:14:16.918] [info] [mongodb] connected to mongodb://localhost:27017 (test.triples).
[14:14:17.119] [info] Using queryable backend with id 'mongodb'.
[14:14:17.119] [info] Using persistent backend with id 'mongodb'.
[14:14:17.121] [info] Loading ontology at '/home/jovyan/workspace/ros/src/knowrob/owl/rdf-schema.xml' with version "Thu Aug  1 11:37:34 2024" and origin "rdf-schema".
[14:14:17.141] [info] Loading ontology at '/home/jovyan/workspace/ros/src/knowrob/owl/owl.rdf' with version "Thu Aug  1 11:37:34 2024" and origin "owl".
[14:14:17.166] [info] Loading ontology at '/home/jovyan/workspace/ros/src/knowrob/owl/test/USD.owl' with version "Tue Aug 20 14:06:43 2024" and origin "USD".
[14:14:17.293] [info] Loading ontology at '/home/jovyan/workspace/ros/src/knowrob/owl/external/DUL.owl' with version "20-08-2024" and origin "DUL".
[14:14:17.540] [info] Loading ontology at '/home/jovyan/workspace/ros/src/knowrob/owl/tes

### Submitting a Query

Now we want to run a query on the loaded semantic map:

In [7]:
phi1 = QueryParser.parse("triple('https://ease-crc.org/ont/USD.owl#house_4', 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#hasPart', ?y)")
resultStream = kb.submitQuery(phi1, QueryContext(QueryFlag.QUERY_FLAG_ALL_SOLUTIONS))
resultQueue = resultStream.createQueue()
# Get the result
nextResult1 = resultQueue.pop_front()

if isinstance(nextResult1, AnswerYes):
    for substitution in nextResult1.substitution():
        variable = substitution[1]
        term = substitution[2]
        print(str(variable) + " : " + str(term))

[14:16:21.516] [warning] an exception occurred for graph query ((DUL:hasPart('https://ease-crc.org/ont/USD.owl#house_4',?y))): BackendError: [mongo] cursor_error: Invalid $set :: caused by :: Unrecognized expression '$sortArray'..
